<a href="https://colab.research.google.com/github/Mrsnellek/MSDS_686_22F8W2/blob/22F8W2/Week_6/Week_6_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Analysis Assingment
## Adapted from Deep Learning with Python (2021)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
# Import the necessary libraries
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers, backend
from tensorflow.keras.layers import TextVectorization, IntegerLookup, Bidirectional, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import string
import re
np.random.seed(1)
import os
import shutil

Download and extract the data.

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

In [ ]:
!tar -xf aclImdb_v1.tar.gz

We will removed the unneeded unsup directory

In [ ]:
!rm -r aclImdb/train/unsup

We will create a validation folder with validation images from the training images


In [ ]:
for category in ("neg", "pos"):
  os.makedirs("aclImdb/val/" + category)
  files = os.listdir("aclImdb/train/" + category)
  np.random.shuffle(files)
  num_val_samples = int(0.2*len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move("aclImdb/train/" + category + "/" + fname,
                "aclImdb/val/" + category + "/" + fname)

In [ ]:
len(files)

Just like with our image data we will use flow from directory so we are not using too much memory

In [ ]:
batch_size = 32
train_flow = keras.utils.text_dataset_from_directory("aclImdb/train", batch_size = batch_size)
val_flow = keras.utils.text_dataset_from_directory("aclImdb/val", batch_size = batch_size)
test_flow = keras.utils.text_dataset_from_directory("aclImdb/test", batch_size = batch_size)


We can inspect the first element of data flowing from the training directory

In [ ]:
for x, y in train_flow:
  print("input shape: ", x.shape)
  print("input type: ", x.dtype)
  print("targets shape: ", y.shape)
  print("targets type: ", y.dtype)
  print("x[0] input text: ", x[0])
  print("y targets: ", y)
  break

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

There is some puncuation and `<br /><br />` values that we should remove.  The below function will remove those features.

In [ ]:
text_vectorization = TextVectorization(
    standardize=custom_standardization)

In [ ]:
check_train = train_flow.map(lambda x, y: (custom_standardization(x), y))

In [ ]:
for x in check_train:
  print("x input text: ", x[0])
  break

Now we will work on vectorizing the x text data.  We will define the number of most commen tokens, words or n-grams, we will use to build our model.  Below we will start with 20000 most common words or n-grams.  This will make sure we are not modeling on spelling mistakes or words that only appear once in the data, in otherwords we do not want to model 'noise'.

In [ ]:
# This removes the y targets.  
text_only_train = train_flow.map(lambda x, y: x)

Now that we have removed the 'y' targets from the data and removed puncuation and breaks we can begin to vectorize the data. We will define how many words or n-grams as the number of tokens.  In our case we will choose 20,000 of the most common words or n-grams.  We will choose to normalize the output using TF-IDF normalization methods.

In [ ]:
max_tokens = 20000
text_vectorization = TextVectorization(
    standardize = custom_standardization,
    ngrams = 2,
    max_tokens = max_tokens,
    output_mode = 'tf_idf'
)

In [ ]:
# This will create a vector that we will use to index our text inputs with values 
# between 0 and 20,000.
text_vectorization.adapt(text_only_train)

In [ ]:
# Appy text_vecorization to the x input text data, but not the y targets
tfid_train = train_flow.map(lambda x, y: (text_vectorization(x), y))
tfid_val = val_flow.map(lambda x, y: (text_vectorization(x), y))
tfid_test = test_flow.map(lambda x, y: (text_vectorization(x), y))

Now we apply the text_vecorization to each of of x text inputs as they flowed from our dirrectories.  Below you can see the ouputs.  The x data is now vectorized and there is normalized TF-IDF value for each word or n-gram found in the text.

In [ ]:
for x, y in tfid_train:
  print("input shape: ", x.shape)
  print("input type: ", x.dtype)
  print("targets shape: ", y.shape)
  print("targets type: ", y.dtype)
  print("x input text: ", x[0])
  print("y target: ", y)
  break

Now we can build our model

In [ ]:
def text_model():
  backend.clear_session()
  model = models.Sequential()
  model.add(layers.Dense(16, activation = 'relu', input_shape = (max_tokens,)))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation = 'sigmoid'))
  model.compile(optimizer = 'rmsprop',
               loss = 'binary_crossentropy',
               metrics = ['accuracy'])
  return model

Now we apply our model to the data.

In [ ]:
model = text_model()
model.fit(tfid_train, validation_data = tfid_val, 
          epochs = 10, 
          callbacks=[EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights = True)])
model.evaluate(tfid_test)

The above model was a bag of words approach with n-grams.  The bag of word approach does not consider the word order with the exeption of the n-grams.  
Now we will add an embedding layer which map word associations into geometric space.  We will also add a LSTM layer for added word order awareness.

Will will truncate the emdeded work space to 600, much more reasonable than 20,000

In [ ]:
max_length = 600
max_tokens = 20000
text_embeding = TextVectorization(
    standardize = custom_standardization,
    max_tokens = max_tokens,
    output_mode = 'int',
    output_sequence_length = max_length
)

In [ ]:
# This will create a vector that we will use to index our text inputs with values 
# between 0 and 20,000.
text_embeding.adapt(text_only_train)

In [ ]:
embed_train = train_flow.map(lambda x, y: (text_embeding(x), y))
embed_val = val_flow.map(lambda x, y: (text_embeding(x), y))
embed_test = test_flow.map(lambda x, y: (text_embeding(x), y))

In [ ]:
for x, y in embed_train:
  print("input shape: ", x.shape)
  print("input type: ", x.dtype)
  print("targets shape: ", y.shape)
  print("targets type: ", y.dtype)
  print("x input text: ", x[0])
  print("y input text: ", y)
  break

In [ ]:
backend.clear_session()
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(max_tokens, 128) (inputs) # Here we will learn the word emdeddings
x = layers.Bidirectional(layers.LSTM(32)) (x)
x = layers.Dropout(0.5) (x)
x = layers.Dense(64, activation="relu") (x)
x = layers.Dropout(0.5) (x)
outputs = layers.Dense(1, activation = 'sigmoid') (x)
model = keras.Model(inputs, outputs)
model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
model.summary()

In [ ]:
model.fit(embed_train, validation_data = embed_val, 
          epochs = 10, 
          callbacks=[EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights = True)])
model.evaluate(embed_test)

Word embedings and LSTM models can  inprove text models. You can also use pretrained word embedings such as [GloVe](https://nlp.stanford.edu/pubs/glove.pdf) or [word2vec](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/rvecs.pdf)  Using these pretained embedings will decrease run time and likely increase accuracy.

Transformer model architecure begain being favored over recuent neural networks, RNN, starting in 2017. Here is a link to the seminal paper that introduced transformer nerual networks: https://arxiv.org/pdf/1706.03762.pdf

We will not cover Transformers this week but you are welcome to explore and utalize transformers in your HW.


# Your turn

Your assignment is to download and perfrom a sentiment classification of Corona Virus tweets.  the [data set](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification) is part of a Kaggle competition.  You will need to download the Kaggle API .json file and upload it to Google Colabs.  I have provided the code you need to download, unzip, and organize the tweets so you can perfrom NLP classification.  

In [ ]:
# Download and innsall Kaggle package
! pip install kaggle -q

In [ ]:
# Make a Kaggle dirrectory
! mkdir ~/.kaggle

#### Upload your kaggle.json API file

In [ ]:
# Copy your kaggle.json file to the newly creadted Kaggle folder
! cp kaggle.json ~/.kaggle/

In [ ]:
# Change permission of the the ,json file.
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the dataset
! kaggle datasets download -d datatattle/covid-19-nlp-text-classification

In [ ]:
# Unzip the dataset
! unzip covid-19-nlp-text-classification.zip

In [ ]:
# Import pandas because file is .csv
import pandas as pd

In [ ]:
# Load the .csv datasets.  View the number of tweets in each file. 
cv_train = pd.read_csv('Corona_NLP_train.csv', encoding='latin-1')
cv_test = pd.read_csv('Corona_NLP_test.csv', encoding='latin-1')
print(len(cv_train))
print(len(cv_test))
print(cv_train.head())
print(cv_train['Sentiment'].unique())

We will simplify the problem by creating 3 classes, positive, neutral, and negative.

In [ ]:
cv_train['Sentiment']=cv_train['Sentiment'].replace('Extremely Positive','Positive')
cv_train['Sentiment']=cv_train['Sentiment'].replace('Extremely Negative','Negative')

cv_test['Sentiment']=cv_test['Sentiment'].replace('Extremely Positive','Positive')
cv_test['Sentiment']=cv_test['Sentiment'].replace('Extremely Negative','Negative')

In [ ]:
# Create CV train and validataion folder
for category in cv_train['Sentiment'].unique():
  os.makedirs("cv/train/" + category)
  os.makedirs("cv/val/" + category)

In [ ]:
# Place each tweet into each folder as a .txt file
i = 0
for category in cv_train['Sentiment'].unique():
  for tweet in cv_train['OriginalTweet'].loc[cv_train['Sentiment'] == category]:
      with open("cv/train/" + category + "/" + str(i) + ".txt", 'w', encoding='utf-8') as my_tweet:
        i +=1
        my_tweet.write(tweet)

In [ ]:
# Split 20% of the training tweets into respective validataion folders.
for category in cv_train['Sentiment'].unique(): 
  files = os.listdir("cv/train/" + category)
  np.random.shuffle(files)
  num_val_samples = int(0.2*len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move("cv/train/" + category + "/" + fname,
                "cv/val/" + category + "/" + fname)

In [ ]:
# Place each test tweet into each folder as a .txt file
for category in cv_test['Sentiment'].unique():
  os.makedirs("cv/test/" + category)
i = 0
for category in cv_test['Sentiment'].unique():
  for tweet in cv_test['OriginalTweet'].loc[cv_test['Sentiment'] == category]:
      with open("cv/test/" + category + "/" + str(i) + ".txt", 'w', encoding='utf-8') as my_tweet:
        i +=1
        my_tweet.write(tweet)

In [ ]:
# Sanitiy check that files are in correct folders.
batch_size = 32
cv_train_flow = keras.utils.text_dataset_from_directory("cv/train", batch_size = batch_size, label_mode = 'categorical')
cv_val_flow = keras.utils.text_dataset_from_directory("cv/val", batch_size = batch_size, label_mode = 'categorical')
cv_test_flow = keras.utils.text_dataset_from_directory("cv/test", batch_size = batch_size, label_mode = 'categorical')

### Continue creating a model to classify the tweets into the 3 different categories.  You can try a pretrained word embeding layer, n-grams, TF-IDF, LSTM, removing stopwords, depth, sequence length, etc. Think about how you will clean the tweets and remove special characters and character artifacts.